# Hypothesis testing problems

## Exercise 1

You are a nutritionist researching two different types of diets to see if there is a significant difference in weight loss after one month. You choose two random groups of people; one group follows the first diet, and the other follows the second. At the end of the month, you record the weight loss (in kg) for each person:

| Diet 1 | Diet 2 |
|:-------|:-------|
| 2.0 | 3.0 |
| 2.5 | 3.2 |
| 3.0 | 3.1 |
| 2.8 | 2.9 |
| 2.3 | 2.8 |
| 2.7 | 3.0 |
| 2.5 | 3.2 |

With these data, it seeks to answer the following question: Is there a significant difference in average weight loss between people who followed the first diet and those who followed the second diet?

To draw conclusions, follow the points below:

- State the hypothesis: null and alternative hypothesis.
- Perform the test to test the hypothesis. You can use a Student's t-test.
- Analyze the conclusions.

In [31]:
# TODO

# Ho: There are no differences in weight loss between the two diets
# Ha: There are differences in weight loss between the two diets

# define groups as lists, n value, and alpha
diet_1 = [2.0, 2.5, 3.0, 2.8, 2.3, 2.7, 2.5]
diet_2 = [3.0, 3.2, 3.1, 2.9, 2.8, 3.0, 3.2]
n = len(diet_1)
alpha = 0.05

n, diet_1, diet_2

(7, [2.0, 2.5, 3.0, 2.8, 2.3, 2.7, 2.5], [3.0, 3.2, 3.1, 2.9, 2.8, 3.0, 3.2])

In [32]:
import numpy as np

#calculate means, standard deviations (1 ddof since both are samples)
diet_1_mean = np.mean(diet_1)
diet_2_mean = np.mean(diet_2)

diet_1_sd = np.std(diet_1, ddof=1)
diet_2_sd = np.std(diet_2, ddof=1)

diet_diff = []

diet_1_mean, diet_1_sd, diet_2_mean, diet_2_sd

(np.float64(2.542857142857143),
 np.float64(0.3309438162646487),
 np.float64(3.0285714285714285),
 np.float64(0.14960264830861927))

In [33]:
# manual calculation of T for two independent samples
T = (diet_1_mean - diet_2_mean) / np.sqrt((diet_1_sd**2/n) + (diet_2_sd**2/n))
T

np.float64(-3.5383407969933938)

In [34]:
from scipy import stats

#calculate ddof of the test (same length for both groups)
ddof = 2*n - 2

#calculate p value from T
p = 2 * (1 - stats.t.cdf(abs(T), 12))
p

np.float64(0.00408327019171395)

In [35]:
from scipy import stats

# check previous calculations with scipy functions
stats.ttest_ind(diet_1, diet_2, alternative='two-sided')

TtestResult(statistic=np.float64(-3.5383407969933938), pvalue=np.float64(0.004083270191713912), df=np.float64(12.0))

In [47]:
!pip install tableone
from tableone import TableOne

Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip is available: 23.1.2 -> 24.3.1
[notice] To update, run: pip install --upgrade pip


In [48]:
#create df to check solution with table one
import pandas as pd

df1 = pd.DataFrame({'values':diet_1})
df1['diet_type'] = 'A'

df2 = pd.DataFrame({'values':diet_2})
df2['diet_type'] = 'B'

df = pd.concat([df1, df2]).reset_index()
df = df.drop(columns='index')
df

,values,diet_type
0,2.0,A
1,2.5,A
2,3.0,A
3,2.8,A
4,2.3,A
5,2.7,A
6,2.5,A
7,3.0,B
8,3.2,B
9,3.1,B


In [49]:
# apply table one to the new dataframe
my_table = TableOne(df, groupby='diet_type', pval=True, htest_name=True)
my_table

Grouped by diet_type                                                            
                                Missing    Overall          A          B P-Value               Test
n                                               14          7          7                           
values, mean (SD)                     0  2.8 (0.4)  2.5 (0.3)  3.0 (0.1)   0.007  Two Sample T-test

In [ ]:
#conclude
if p > alpha:
    print("'Ho' can't be rejected: There is not enough evidence to affirm that the two diets are different")
elif p < alpha:
    print("'Ho' is rejected and 'Ha' is accepted: there are differences in the diets")

## ANOVA

**ANOVA** (*Analysis of Variance*) is a statistical technique used to compare the measures of two or more groups. The idea behind ANOVA is to decompose the total variability in the data into two components: between-group variability and within-group variability:

- **Between-group variability**: This variability refers to the differences between the group means. If this variability is considerably larger than the within-group variability, it could be an indication that at least one of the group means is different.
- **Within-group variability**: This variability refers to the dispersion of the data within each group. If all groups have similar variability, then any noticeable difference in group means could be considered significant.

Hypotheses in ANOVA typically include:

- **Null hypothesis** ($H₀$): The means of all groups are equal.
- **Alternative hypothesis** ($H₁$): At least one of the group means is different.

If the ANOVA test result is significant (e.g., a p-value less than a threshold such as 0.05), this suggests that at least one group mean is different.

## Exercise 2

A farmer decides to test three different types of fertilizers to determine if one is superior in terms of corn production. The farmer plants corn on 15 identical plots and uses all three fertilizers (5 plots for each type). At the end of the season, he measures the corn yield (in kg) of each plot, with the following result:

| Fertilizer 1 | Fertilizer 2 | Fertilizer 3 |
|:-------------|:-------------|:-------------|
| 20 | 22 | 24 |
| 21 | 21 | 23 |
| 20 | 23 | 22 |
| 19 | 22 | 23 |
| 20 | 21 | 24 |

With this data, he seeks to answer the following question: Is there a significant difference in average corn yield between the three types of fertilizers?

To help you, follow the points below:

- State the hypothesis: null and alternative hypothesis.
- Perform the ANOVA test.
- Analyze the conclusions.
- If one fertilizer is better than another, how can we know it?

In [50]:
# TODO
# Ho: There are no significant differences in the mean corn yields between the fertilizers.
# Ha: There is at least one significant difference in the mean corn yields between the fertilizers.

# define groups as lists, n value, and alpha
fert_1 = [20, 21, 20, 19, 20]
fert_2 = [22, 21, 23, 22, 21]
fert_3 = [24, 23, 22, 23, 24]

n = len(fert_1)
N = n*3
k = 3
alpha = 0.05

N

15

In [51]:
import numpy as np
from scipy import stats

#calculate means, standard deviations (1 ddof since all groups are samples), and standard errors
fert_1_mean = np.mean(fert_1)
fert_2_mean = np.mean(fert_2)
fert_3_mean = np.mean(fert_3)

fert_1_sd = np.std(fert_1, ddof=1)
fert_2_sd = np.std(fert_2, ddof=1)
fert_3_sd = np.std(fert_3, ddof=1)

fert_1_se = stats.sem(fert_1, ddof=1)
fert_2_se = stats.sem(fert_2, ddof=1)
fert_3_se = stats.sem(fert_3, ddof=1)


fert_1_mean, fert_1_sd, fert_1_se, fert_2_mean, fert_2_sd, fert_2_se, fert_3_mean, fert_3_sd, fert_3_se

(np.float64(20.0),
 np.float64(0.7071067811865476),
 np.float64(0.31622776601683794),
 np.float64(21.8),
 np.float64(0.8366600265340756),
 np.float64(0.3741657386773941),
 np.float64(23.2),
 np.float64(0.8366600265340756),
 np.float64(0.3741657386773941))

In [52]:
#calculate ddofs
bg_ddof = k-1
wg_ddof = N-k
total_ddof = N-1

#calculate sum of squares between groups, within groups and total
global_mean = (fert_1_mean+fert_2_mean+fert_3_mean)/k

ss_bg = n*((fert_1_mean - global_mean)**2 + (fert_2_mean - global_mean)**2 + (fert_3_mean - global_mean)**2)
ss_bg

ss_wg = 0

for i in range(len(fert_1)):
    a = ((fert_1[i] - fert_1_mean)**2)  + ((fert_2[i] - fert_2_mean)**2) + ((fert_3[i] - fert_3_mean)**2)
    ss_wg = ss_wg + a

ss_total = ss_bg + ss_wg

#calculate mean squares between and within groups
ms_bg = ss_bg/(k-1)
ms_wg = ss_wg/(N-k)

#calculate value of F-Statistic
F = ms_bg/ms_wg
F

np.float64(20.3157894736842)

In [67]:
# compare results with f_oneway formula from scipy stats
F, p = stats.f_oneway(fert_1, fert_2, fert_3)
F, p

(np.float64(20.315789473684188), np.float64(0.00014047824793190475))

In [62]:
#prepare df for tableone check
df1 = pd.DataFrame({"values":fert_1})
df1['fertilizer'] = 'F1'
df2 = pd.DataFrame({"values":fert_2})
df2['fertilizer'] = 'F2'
df3 = pd.DataFrame({"values":fert_3})
df3['fertilizer'] = 'F3'

df = pd.concat([df1, df2, df3])
df = df.reset_index()
df_fertilizers = df.drop(columns='index')
df_fertilizers

,values,fertilizer
0,20,F1
1,21,F1
2,20,F1
3,19,F1
4,20,F1
5,22,F2
6,21,F2
7,23,F2
8,22,F2
9,21,F2


In [65]:
my_table_2 = TableOne(df_fertilizers, groupby='fertilizer', pval=True, htest_name=True)
my_table_2

Grouped by fertilizer                                                                       
                                 Missing     Overall          F1          F2          F3 P-Value           Test
n                                                 15           5           5           5                       
values, mean (SD)                      0  21.7 (1.5)  20.0 (0.7)  21.8 (0.8)  23.2 (0.8)  <0.001  One-way ANOVA

In [69]:
#conclude
if p > alpha:
    print("'Ho' can't be rejected: There is not enough evidence to affirm that the three fertilizers cause different crop yield means")
elif p < alpha:
    print("'Ho' is rejected and 'Ha' is accepted: There are statistically significant differences to affirm that the three fertilizers cause different crop yield means")

'Ho' is rejected and 'Ha' is accepted: There are statistically significant differences to affirm that the three fertilizers cause different crop yield means
